In [155]:
import numpy as np
from scipy import linalg, special, sparse
import sympy
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import cvxpy as cp
import copy
import requests
import xmltodict


sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

# Import simulation output

In [94]:
time = '10'
date = '2023-03-08'
experiment = 'balance-fix'
entry = f'{experiment}_{time}_{date}'
folder = f'out/fbagd/{entry}/'

In [95]:
output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [96]:
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [97]:
stoichiometry = agent['ecoli-metabolism'].model.stoichiometry
maintenance_reaction = agent['ecoli-metabolism'].model.maintenance_reaction
stoichiometry["maintenance_reaction"] = maintenance_reaction

bad_rxns = ["RXN-12440", "TRANS-RXN-121", "TRANS-RXN-300"]
for rxn in bad_rxns:
    stoichiometry.pop(rxn, None)


In [98]:
stoichiometry['1.1.1.127-RXN']

{'2-DEHYDRO-3-DEOXY-D-GLUCONATE[c]': -1,
 'NAD[c]': -1,
 'CPD-343[c]': 1,
 'NADH[c]': 1,
 'PROTON[c]': 1}

Pandas automatically understands dicts of dicts as matrices. 

# Change nutrient composition

In [185]:
uptake = (fba["unconstrained_molecules"][3]).copy()
uptake_c = fba["constrained_molecules"]

uptake.extend(list(uptake_c.keys()))
uptake = set(uptake)

# addition = set(["CPD0-1074[c]"])
addition = set(
    [
        # "D-Ribopyranose[p]",
        # "NITRATE[p]"
        # 'THIAMINE[p]',
        'HYDROGEN-MOLECULE[p]',
        'FUM[p]'
    ]
)
removals = set(
                [
                  "GLC[p]",
                    # 'AMMONIUM[c]',
                    # 'WATER[p]'
                     # 'Pi[p]',
                     # 'SULFATE[p]'
               'OXYGEN-MOLECULE[p]'
                ]
                )


uptake = uptake | addition
uptake = uptake - removals
uptake

{'AMMONIUM[c]',
 'CA+2[p]',
 'CARBON-DIOXIDE[p]',
 'CL-[p]',
 'CO+2[p]',
 'FE+2[p]',
 'FUM[p]',
 'HYDROGEN-MOLECULE[p]',
 'K+[p]',
 'L-SELENOCYSTEINE[c]',
 'MG+2[p]',
 'MN+2[p]',
 'NA+[p]',
 'NI+2[p]',
 'Pi[p]',
 'SULFATE[p]',
 'WATER[p]',
 'ZN+2[p]'}

In [186]:
Sd = pd.DataFrame(stoichiometry, dtype=np.int8).fillna(0).astype(np.int8)


In [187]:
#stoich_dict = {item["reaction id"]: item["stoichiometry"] for item in stoichiometry}

mets_df = pd.DataFrame(Sd.index, columns=["Name"]) # index to name
rxns_df = pd.DataFrame(Sd.columns, columns=["Name"]) 

exchanges = pd.DataFrame(fba["estimated_exchange_dmdt"])
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"])
maintenance = pd.DataFrame(fba["target_maintenance_flux"])

kinetic = pd.DataFrame(fba["target_kinetic_fluxes"])
kin_rxns = list(kinetic.keys())

homeostatic_mets = list(homeostatic.keys())
exchange_mets = list(exchanges.keys())
intermediate_mets = list(set(mets_df["Name"]) - set(homeostatic.keys()))

n_mets, n_orig_rxns = Sd.shape

exchange_mets.extend(list(uptake))

Se = pd.DataFrame(index=Sd.index)

for met in exchange_mets:
    exch_name = met + " exchange"
    if met in uptake:
        Se[exch_name] = np.zeros(n_mets).astype(np.int8)
        Se.at[met, exch_name] = 1
    
    Se[exch_name + " rev"] = np.zeros(n_mets).astype(np.int8)
    Se.at[met, exch_name + " rev"] = -1
      
    
    
S_orig = np.array(Sd, dtype=np.float64)
S_exch = np.array(Se, dtype=np.float64)

_, n_exch_rxns = S_exch.shape


int_df = mets_df[mets_df['Name'].isin(intermediate_mets)].reset_index()
int_idx = list(int_df["index"])

hom_df = mets_df[mets_df['Name'].isin(homeostatic_mets)].reset_index()
target_hom = pd.DataFrame(homeostatic.loc[4, :]).reset_index(drop=False)
target_hom.columns = ["Name", "Count"]
merge_df = hom_df.set_index('Name').join(target_hom.set_index('Name'))

kin_df = rxns_df[rxns_df['Name'].isin(kin_rxns)].reset_index()
target_kin = pd.DataFrame(kinetic.loc[4, :]).reset_index(drop=False)
target_kin.columns = ["Name", "Flux"]
merge_kin_df = kin_df.set_index('Name').join(target_kin.set_index('Name'))

v_hom = np.array(merge_df["Count"]).astype(np.float64)
hom_idx = np.array(merge_df["index"]).astype(np.int64)

v_kin = np.array(merge_kin_df["Flux"]).astype(np.float64)
kin_idx = np.array(merge_kin_df["index"]).astype(np.int64)

v_main = np.array(fba["target_maintenance_flux"][4])
main_idx = list(Sd.columns).index("maintenance_reaction")


secretion_idx = np.where(S_exch.sum(axis=0) == -1)[0]

In [188]:
v = cp.Variable(n_orig_rxns)
e = cp.Variable(n_exch_rxns)
v_t = cp.hstack([v,e])
dm = S_orig @ v + S_exch @ e 


constr = []
constr.append(dm[int_idx] == 0)
constr.append(v[main_idx] == v_main)
# constr.append(dm[hom_idx] == v_hom)
constr.extend([v >= 0, v <= 100000000, e >= 0, e <= 100000000])

loss = 0
loss += cp.norm1(dm[hom_idx] - v_hom)
loss += 0.001*(cp.sum(e[secretion_idx]))
loss += 0.00001*(cp.sum(v))
loss += 0.000001 * cp.norm1(v[kin_idx] - v_kin)

In [189]:
p = cp.Problem(
        cp.Minimize(loss),
    constr
    )
               
p.solve(solver=cp.GLOP, verbose=False)

13480.964595513162

# Analyze output

In [190]:
exchange_rxns = Se.columns
pd.DataFrame({exchange_rxns[i]: [e.value[i]] for i in range(len(exchange_rxns))}).sort_values(by=0, axis=1, ascending=False).T.head(20)

,0
FUM[p] exchange,1.110080e+07
HYDROGEN-MOLECULE[p] exchange,9.624735e+06
SUC[p] exchange rev,9.560213e+06
WATER[p] exchange,8.099473e+06
FORMATE[p] exchange rev,2.317448e+06
AMMONIUM[c] exchange,8.765715e+05
CARBON-DIOXIDE[p] exchange rev,4.605137e+05
Pi[p] exchange,9.788700e+04
K+[p] exchange,1.742400e+04
SULFATE[p] exchange,6.256000e+03


In [191]:
exchanges.loc[4, :].abs().sort_values().tail(10)

MG+2[p]                   774
SULFATE[p]               6263
K+[p]                   17424
Pi[p]                  100667
PROTON[p]              775061
AMMONIUM[c]            887279
GLC[p]                 897920
OXYGEN-MOLECULE[p]    1733230
CARBON-DIOXIDE[p]     1873581
WATER[p]              3625912
Name: 4, dtype: int64

In [192]:
flux_indexed = list(Sd.columns[:n_orig_rxns])
fluxes = pd.DataFrame({flux_indexed[i]: [v.value[i], stoichiometry[flux_indexed[i]]] for i in range(len(flux_indexed))}).T
fluxes.sort_values(by=0, axis=0, ascending=False).head(15)

,0,1
R601-RXN-FUM/REDUCED-MENAQUINONE//SUC/CPD-9728.38.,9657218.583942,"{'FUM[c]': -1, 'REDUCED-MENAQUINONE[i]': -1, '..."
RXN0-5256-CPD-9728/PROTON/HYDROGEN-MOLECULE//REDUCED-MENAQUINONE/PROTON.62.,9624734.583942,"{'CPD-9728[i]': -1, 'PROTON[c]': -2, 'HYDROGEN..."
TRANS-RXN-106,9560212.583942,"{'SUC[c]': -1, 'FUM[p]': -1, 'FUM[c]': 1, 'SUC..."
TRANS-RXN-145,8099473.0,"{'WATER[p]': -1, 'WATER[c]': 1}"
ATPSYN-RXN (reverse),4011746.604471,"{'ATP[c]': 1, 'WATER[c]': 1, 'PROTON[c]': 3, '..."
TRANS-RXN-1,2317447.75,"{'FORMATE[c]': -1, 'FORMATE[p]': 1}"
PYRUVFORMLY-RXN (reverse),1920461.875,"{'FORMATE[c]': 1, 'ACETYL-COA[c]': 1, 'PYRUVAT..."
maintenance_reaction,1898967.604471,"{'ATP[c]': -1, 'WATER[c]': -1, 'ADP[c]': 1, 'P..."
FLAVONADPREDUCT-RXN (reverse),1637375.875,"{'Reduced-flavodoxins[c]': 1, 'NADP[c]': 1, 'O..."
PYFLAVOXRE-RXN (reverse),1637048.875,"{'PYRUVATE[c]': 1, 'Oxidized-flavodoxins[c]': ..."


In [193]:
sim_fluxes = pd.DataFrame(fba["estimated_fluxes"])
pd.DataFrame(sim_fluxes.loc[4, :].abs().sort_values(ascending=False).head(10))

,4
ATPSYN-RXN (reverse),1509759
TRANS-RXN0-547[CCO-PM-BAC-NEG]-WATER//WATER.29.,905893
SUCCINATE-DEHYDROGENASE-UBIQUINONE-RXN-SUC/UBIQUINONE-8//FUM/CPD-9956.31.,857780
R601-RXN-FUM/REDUCED-MENAQUINONE//SUC/CPD-9728.38.,700754
RXN0-5388-CPD-9728/NADH/PROTON//REDUCED-MENAQUINONE/NAD/PROTON.53.,699206
TRANS-RXN0-545[CCO-PM-BAC-NEG]-CARBON-DIOXIDE//CARBON-DIOXIDE.47. (reverse),468395
FUMHYDR-RXN__FUMARASE-A,449687
TRANS-RXN0-474,433303
RXN-21817-CPD-9956/PROTON/OXYGEN-MOLECULE//UBIQUINONE-8/PROTON/WATER.59.,433255
GAPOXNPHOSPHN-RXN,384822


In [194]:
sim_fluxes = pd.DataFrame(fba["estimated_fluxes"])
pd.DataFrame(sim_fluxes.loc[4, :].abs().sort_values(ascending=False).head(10))

,4
ATPSYN-RXN (reverse),1509759
TRANS-RXN0-547[CCO-PM-BAC-NEG]-WATER//WATER.29.,905893
SUCCINATE-DEHYDROGENASE-UBIQUINONE-RXN-SUC/UBIQUINONE-8//FUM/CPD-9956.31.,857780
R601-RXN-FUM/REDUCED-MENAQUINONE//SUC/CPD-9728.38.,700754
RXN0-5388-CPD-9728/NADH/PROTON//REDUCED-MENAQUINONE/NAD/PROTON.53.,699206
TRANS-RXN0-545[CCO-PM-BAC-NEG]-CARBON-DIOXIDE//CARBON-DIOXIDE.47. (reverse),468395
FUMHYDR-RXN__FUMARASE-A,449687
TRANS-RXN0-474,433303
RXN-21817-CPD-9956/PROTON/OXYGEN-MOLECULE//UBIQUINONE-8/PROTON/WATER.59.,433255
GAPOXNPHOSPHN-RXN,384822


In [195]:
dmdt_indexed = list(Sd.index)
# hom_idx
dmdt_dict = {dmdt_indexed[i]: [dm.value[i], v_hom[j], dm.value[i] - v_hom[j]] for j, i in enumerate(hom_idx)}
dmdt = pd.DataFrame(dmdt_dict).sort_values(by=0, axis=1).T
dmdt
#dmdt = pd.DataFrame({dmdt_indexed[i]: [dm.value[i], v_hom[j]] for j, i in enumerate(hom_idx)})
#dmdt.sort_values(by=0, axis=1, ascending=False).T.head(20)

,0,1,2
PPI[c],-78147.0,-78147.0,-1.455192e-11
PROTON[c],-17999.0,-17999.0,1.280569e-08
GMP[c],-4851.0,-4851.0,3.637979e-12
AMP[c],-4531.0,-4531.0,-2.910383e-11
CMP[c],-4307.0,-4307.0,0.000000e+00
...,...,...,...
VAL[c],34446.0,34446.0,0.000000e+00
GLY[c],34610.0,34610.0,5.820766e-11
LEU[c],36085.0,36085.0,0.000000e+00
L-ALPHA-ALANINE[c],43678.0,43678.0,0.000000e+00


In [196]:
mets_df[mets_df.loc[:, "Name"].str.contains("FUM")]

,Name
576,FUM[c]
3678,FUM[p]
4506,FUM[e]


In [197]:
dmdt[dmdt.index.str.contains('HYDROGEN')]

,0,1,2


# Set up schema for testing lots of fluxes

## Connect to EcoCyc to go from Genes -> Reactions

In [198]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})

<Response [200]>

In [199]:
r = s.get('https://websvc.biocyc.org/getxml?id=ECOLI:RXN-12440&detail=low&fmt=json')
rxn = xmltodict.parse(r.content)
#rxn['ptools-xml']['Reaction']['right']

In [228]:
condition_sets = {0: {"Genes": set(["EG10468", "EG10469", "EG10470", "EG11799", "EG11800", "EG11801", "G7554"]), 
                      "Add": set(["FUM[p]", 'HYDROGEN-MOLECULE[p]']),
                      "Remove": set(["GLC[p]", "OXYGEN-MOLECULE[p]"]),
                      "Environment": "Hydrogen + fumarate, -O2, -Glc"},
                  1: {"Genes": set(["EG10814", "EG10815", "EG10816", "EG10818"]), 
                      "Add": set(["D-Ribopyranose[p]"]),
                      "Remove": set(["GLC[p]"]),
                      "Environment": "Ribose C-source"},
                  2: {"Genes": set(["EG11574", "G6566", "M007", "EG11573", "EG11572"]), 
                      "Add": set(["THIAMINE[p]"]),
                      "Remove": set(),
                      "Environment": "Thiamine salvaging"},
                  3: {"Genes": set(["EG10638", "EG10639", "EG10640", "EG10644", "EG10647", "EG10648"]), 
                      "Add": set(["NITRATE[p]"]),
                      "Remove": set(["OXYGEN-MOLECULE[p]"]),
                      "Environment": "Nitrate as reduction target"}
                 }

condition_df = pd.DataFrame(condition_sets).T
condition_df["Reactions"] = [[np.nan]] * len(condition_df)
condition_df

,Genes,Add,Remove,Environment,Reactions
0,"{EG10470, EG11801, G7554, EG10469, EG11800, EG...","{FUM[p], HYDROGEN-MOLECULE[p]}","{GLC[p], OXYGEN-MOLECULE[p]}","Hydrogen + fumarate, -O2, -Glc",[nan]
1,"{EG10818, EG10816, EG10814, EG10815}",{D-Ribopyranose[p]},{GLC[p]},Ribose C-source,[nan]
2,"{G6566, M007, EG11574, EG11573, EG11572}",{THIAMINE[p]},{},Thiamine salvaging,[nan]
3,"{EG10638, EG10639, EG10640, EG10647, EG10648, ...",{NITRATE[p]},{OXYGEN-MOLECULE[p]},Nitrate as reduction target,[nan]


In [236]:
for i in range(len(condition_df.index)):
    
    genes = condition_df.loc[i, "Genes"]
    rxn_set = set()

    for gene in genes:
        function = "reactions-of-gene"
        name = gene
        req_str = f"https://websvc.biocyc.org/apixml?fn={function}&id=ECOLI:{name}&detail=none&fmt=json"

        r = s.get(req_str)
        o = xmltodict.parse(r.content)['ptools-xml']['Reaction']
        if type(o) is dict:
            o = [o]
        
        for rxn in o:
            if type(rxn) is not str: # no rxns
                rxn_set.add(rxn['@frameid'])
            else:
                print(gene, rxn)
            
    condition_df.at[i, "Reactions"] = rxn_set

In [238]:
condition_df

,Genes,Add,Remove,Environment,Reactions
0,"{EG10470, EG11801, G7554, EG10469, EG11800, EG...","{FUM[p], HYDROGEN-MOLECULE[p]}","{GLC[p], OXYGEN-MOLECULE[p]}","Hydrogen + fumarate, -O2, -Glc","{RXN0-5256, RXN0-7399, RXN-16420}"
1,"{EG10818, EG10816, EG10814, EG10815}",{D-Ribopyranose[p]},{GLC[p]},Ribose C-source,"{RIBOKIN-RXN, ABC-28-RXN}"
2,"{G6566, M007, EG11574, EG11573, EG11572}",{THIAMINE[p]},{},Thiamine salvaging,"{ABC-32-RXN, RXN0-7298, THIKIN-RXN, THIAZOLSYN..."
3,"{EG10638, EG10639, EG10640, EG10647, EG10648, ...",{NITRATE[p]},{OXYGEN-MOLECULE[p]},Nitrate as reduction target,"{RXN0-3501, RXN-15119, RXN0-7124}"


## Initialize FBA

In [242]:
#stoich_dict = {item["reaction id"]: item["stoichiometry"] for item in stoichiometry}

mets_df = pd.DataFrame(Sd.index, columns=["Name"]) # index to name
rxns_df = pd.DataFrame(Sd.columns, columns=["Name"]) 

exchanges = pd.DataFrame(fba["estimated_exchange_dmdt"])
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"])
maintenance = pd.DataFrame(fba["target_maintenance_flux"])

kinetic = pd.DataFrame(fba["target_kinetic_fluxes"])
kin_rxns = list(kinetic.keys())

homeostatic_mets = list(homeostatic.keys())
exchange_mets = list(exchanges.keys())
intermediate_mets = list(set(mets_df["Name"]) - set(homeostatic.keys()))

n_mets, n_orig_rxns = Sd.shape


int_df = mets_df[mets_df['Name'].isin(intermediate_mets)].reset_index()
int_idx = list(int_df["index"])

hom_df = mets_df[mets_df['Name'].isin(homeostatic_mets)].reset_index()
target_hom = pd.DataFrame(homeostatic.loc[4, :]).reset_index(drop=False)
target_hom.columns = ["Name", "Count"]
merge_df = hom_df.set_index('Name').join(target_hom.set_index('Name'))

kin_df = rxns_df[rxns_df['Name'].isin(kin_rxns)].reset_index()
target_kin = pd.DataFrame(kinetic.loc[4, :]).reset_index(drop=False)
target_kin.columns = ["Name", "Flux"]
merge_kin_df = kin_df.set_index('Name').join(target_kin.set_index('Name'))

v_hom = np.array(merge_df["Count"]).astype(np.float64)
hom_idx = np.array(merge_df["index"]).astype(np.int64)

v_kin = np.array(merge_kin_df["Flux"]).astype(np.float64)
kin_idx = np.array(merge_kin_df["index"]).astype(np.int64)

v_main = np.array(fba["target_maintenance_flux"][4])
main_idx = list(Sd.columns).index("maintenance_reaction")

## Run FBA for each cond.

In [266]:
condition_idx = 1

uptake = (fba["unconstrained_molecules"][3]).copy()
uptake_c = fba["constrained_molecules"]

uptake.extend(list(uptake_c.keys()))
uptake = set(uptake)

addition = condition_df.at[condition_idx, "Add"]
removals = condition_df.at[condition_idx, "Remove"]

uptake = uptake | addition
uptake = uptake - removals
uptake

{'AMMONIUM[c]',
 'CA+2[p]',
 'CARBON-DIOXIDE[p]',
 'CL-[p]',
 'CO+2[p]',
 'D-Ribopyranose[p]',
 'FE+2[p]',
 'K+[p]',
 'L-SELENOCYSTEINE[c]',
 'MG+2[p]',
 'MN+2[p]',
 'NA+[p]',
 'NI+2[p]',
 'OXYGEN-MOLECULE[p]',
 'Pi[p]',
 'SULFATE[p]',
 'WATER[p]',
 'ZN+2[p]'}

In [271]:
# just need to change exchanges.
exchange_mets = list(exchanges.keys())
exchange_mets.extend(list(uptake))

Se = pd.DataFrame(index=Sd.index)

for met in exchange_mets:
    exch_name = met + " exchange"
    if met in uptake:
        Se[exch_name] = np.zeros(n_mets).astype(np.int8)
        Se.at[met, exch_name] = 1
    
    Se[exch_name + " rev"] = np.zeros(n_mets).astype(np.int8)
    Se.at[met, exch_name + " rev"] = -1
      
S_exch = np.array(Se, dtype=np.float64)

_, n_exch_rxns = S_exch.shape

secretion_idx = np.where(S_exch.sum(axis=0) == -1)[0]

In [272]:
v = cp.Variable(n_orig_rxns)
e = cp.Variable(n_exch_rxns)
v_t = cp.hstack([v,e])
dm = S_orig @ v + S_exch @ e 


constr = []
constr.append(dm[int_idx] == 0)
constr.append(v[main_idx] == v_main)
# constr.append(dm[hom_idx] == v_hom)
constr.extend([v >= 0, v <= 100000000, e >= 0, e <= 100000000])

loss = 0
loss += cp.norm1(dm[hom_idx] - v_hom)
loss += 0.001*(cp.sum(e[secretion_idx]))
loss += 0.00001*(cp.sum(v))
loss += 0.000001 * cp.norm1(v[kin_idx] - v_kin)

p = cp.Problem(
        cp.Minimize(loss),
    constr
    )
               
p.solve(solver=cp.GLOP, verbose=False)

1675.9394051122542

In [273]:
# Check flux
flux_indexed = list(Sd.columns[:n_orig_rxns])
fluxes = pd.DataFrame({flux_indexed[i]: [v.value[i], stoichiometry[flux_indexed[i]]] for i in range(len(flux_indexed))}).T
fluxes.columns = ["Flux", "Stoichiometry"]

relevant_rxns = condition_df.at[condition_idx, "Reactions"]
relevant_rxn_idx = []

for rxn in relevant_rxns:
    relevant_rxn_idx.extend(list(np.where(fluxes.index.str.contains(rxn))[0] ))
    
fluxes.iloc[relevant_rxn_idx, :]

,Flux,Stoichiometry
RIBOKIN-RXN,-0.0,"{'D-Ribofuranose[c]': -1, 'ATP[c]': -1, 'CPD-1..."
RIBOKIN-RXN-CPD-10330/ATP//CPD-15317/ADP/PROTON.36.,-0.0,"{'CPD-10330[c]': -1, 'ATP[c]': -1, 'CPD-15317[..."
RIBOKIN-RXN-CPD-10330/ATP//CPD-15318/ADP/PROTON.36.,-0.0,"{'CPD-10330[c]': -1, 'ATP[c]': -1, 'CPD-15318[..."
RIBOKIN-RXN-CPD-10330/ATP//CPD-16551/ADP/PROTON.36.,0.0,"{'CPD-10330[c]': -1, 'ATP[c]': -1, 'CPD-16551[..."
RIBOKIN-RXN-CPD0-1108/ATP//CPD-15317/ADP/PROTON.36.,-0.0,"{'CPD0-1108[c]': -1, 'ATP[c]': -1, 'CPD-15317[..."
RIBOKIN-RXN-CPD0-1108/ATP//CPD-15318/ADP/PROTON.36.,300342.937169,"{'CPD0-1108[c]': -1, 'ATP[c]': -1, 'CPD-15318[..."
RIBOKIN-RXN-CPD0-1108/ATP//CPD-16551/ADP/PROTON.36.,647242.874338,"{'CPD0-1108[c]': -1, 'ATP[c]': -1, 'CPD-16551[..."
RIBOKIN-RXN-D-Ribofuranose/ATP//CPD-15318/ADP/PROTON.41.,-0.0,"{'D-Ribofuranose[c]': -1, 'ATP[c]': -1, 'CPD-1..."
RIBOKIN-RXN-D-Ribofuranose/ATP//CPD-16551/ADP/PROTON.41.,-0.0,"{'D-Ribofuranose[c]': -1, 'ATP[c]': -1, 'CPD-1..."
ABC-28-RXN,-0.0,"{'ATP[c]': -1, 'D-Ribopyranose[p]': -1, 'WATER..."


In [ ]:
# Check exchange

# Troubleshoot wrong fluxes

In [112]:
top_wrong = fluxes_wrong.sort_values(by=0, axis=1, ascending=False).T.head(50)
top_right = fluxes.sort_values(by=0, axis=1, ascending=False).T.head(50)

top_wrong.join(top_right, lsuffix="wrong", rsuffix="right")

NameError: name 'fluxes_wrong' is not defined

In [ ]:
m2 = []

for rxn in ["LXULRU5P-RXN (reverse)", "RXN-21400", "RXN0-703 (reverse)", "RXN0-704", "RIBULPEPIM-RXN__EG12287-MONOMER", "RXN0-705__EG12285-MONOMER"]:
    m2.append({key: 2*value for key, value in stoichiometry[rxn].items()})
    
pd.DataFrame(m2, dtype=np.int8).fillna(0).astype(np.int8).sum(axis=0)

In [ ]:
m1 = []

for rxn in ["ALANINE-AMINOTRANSFERASE-RXN (reverse)", "MALATE-DEH-RXN (reverse)", "RXN-3523", "RXN-10981", \
 "RXN-12440", "CHD-RXN (reverse)", "RXN0-7230-CHOLINE/UBIQUINONE-8//BETAINE_ALDEHYDE/CPD-9956.48.", "ALARACECAT-RXN", "PMPOXI-RXN", "RXN0-5240"]:
    m1.append({key: value for key, value in stoichiometry[rxn].items()})
    
pd.DataFrame(m1, dtype=np.int8).fillna(0).astype(np.int8).sum(axis=0)

# Extract specific fluxes for balancing

In [ ]:
rxns_names = ['PGLUCISOM-RXN', '6PFRUCTPHOS-RXN', 'F16ALDOLASE-RXN', 'TRIOSEPISOMERIZATION-RXN', 
              'GAPOXNPHOSPHN-RXN', 'PHOSGLYPHOS-RXN', '3PGAREARR-RXN',
              '2PGADEHYDRAT-RXN', 'PEPDEPHOS-RXN']

In [ ]:
fluxes.sort_values(by=0, axis=1, ascending=False).T.head(10)

In [ ]:
fluxes_small = fluxes.loc[fluxes.index.isin(rxns_names), :]
fluxes_small

In [ ]:
fluxes.loc[fluxes.index.str.contains("TRIOSEPISOMERIZATION-RXN"),:]

,0,1
TRIOSEPISOMERIZATION-RXN,-0.0,"{'GAP[c]': -1, 'DIHYDROXY-ACETONE-PHOSPHATE[c]..."
TRIOSEPISOMERIZATION-RXN (reverse),649460.771914,"{'GAP[c]': 1, 'DIHYDROXY-ACETONE-PHOSPHATE[c]'..."


In [524]:
fluxes.loc[fluxes.index.str.contains("3PGAREARR-RXN"),:]

,0,1
3PGAREARR-RXN,0.0,"{'2-PG[c]': -1, 'G3P[c]': 1}"
3PGAREARR-RXN (reverse),1120335.543829,"{'2-PG[c]': 1, 'G3P[c]': -1}"


In [525]:
fluxes.loc[fluxes.index.str.contains("PHOSGLYPHOS-RXN"),:]

,0,1
PHOSGLYPHOS-RXN,-0.0,"{'G3P[c]': -1, 'ATP[c]': -1, 'DPG[c]': 1, 'ADP..."
PHOSGLYPHOS-RXN (reverse),1251221.543829,"{'G3P[c]': 1, 'ATP[c]': 1, 'DPG[c]': -1, 'ADP[..."


In [526]:
fluxes.loc[fluxes.index.str.contains("2TRANSKETO-RXN"),:]

,0,1
2TRANSKETO-RXN,-0.0,"{'ERYTHROSE-4P[c]': -1, 'XYLULOSE-5-PHOSPHATE[..."
2TRANSKETO-RXN (reverse),49000.0,"{'ERYTHROSE-4P[c]': 1, 'XYLULOSE-5-PHOSPHATE[c..."


In [527]:
fluxes.columns.isin(rxns_names)

array([False, False])